<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/Kaggle/2022Q2/Kaggle_2Q2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de datos

In [6]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

In [2]:
import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c ap-tp2-2q2022


import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

Mounted at /content/drive
 96% 49.0M/51.3M [00:01<00:00, 37.7MB/s]
100% 51.3M/51.3M [00:01<00:00, 36.8MB/s]


In [19]:
db=pd.read_csv("origen.csv", sep=",")

# Regresión lineal

In [5]:
X=['numVotes','startYear','runtimeMinutes']

In [9]:
reg = LinearRegression().fit(db[X], db['averageRating'])
reg.score(db[X], db['averageRating'])

0.024976875345017358

# Salida

In [11]:
res=pd.read_csv("testear.csv", sep=",")

In [12]:
resultado=reg.predict(res[X])

In [16]:
salida = pd.DataFrame(data={"averageRating": resultado}).astype(str)
salida.index = res.Id
salida.to_csv("Linear_Reg.csv", sep=',',index=True,  index_label='Id')

# Extra Trees

In [17]:
from sklearn.ensemble import ExtraTreesRegressor

In [20]:
reg = ExtraTreesRegressor().fit(db[X], db['averageRating'])
reg.score(db[X], db['averageRating'])

0.506763341745001

# Train Test Split

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(db[X], db['averageRating'], test_size=0.2, random_state=10)

In [23]:
reg = ExtraTreesRegressor().fit(X_train, y_train)
reg.score(X_test,  y_test)

0.062153643265038805

# Regularizar

In [24]:
reg = ExtraTreesRegressor(max_depth=20).fit(X_train, y_train)
reg.score(X_test,  y_test)

0.1578344590844627

¿Qué sigue? ¿Cómo mejorar? 

- Hicimos una regularización y el modelo mejoró, pero eso no quiere decir que el modelo esté optimizado. Habría que seguir probando, incluso convendría hacerlo por Cross Validation. 
- Solo regularizamos con max_depth. Extra Trees tiene otros hiperparámetros que podrían mejorar la performance. Conocer los modelos ayuda a tener mejor intuición de que es lo que puede pasar al tocar cada hiperparámetro, pero nada me garantiza resultados: hay que probar. 
- Probamos con Extra Trees que es uno de los modelos que mejor funcionan. Sin embargo, hay muchos otros. Casi con seguridad Catboost y XGBoost funcionan mejor. Hay que probarlos. 
- Todavía el data set tiene mucha información para incorporar. ¡Estamos usando solo tres features! Hay que limpiar e incorporar las demás. Por más que usemos el mejor modelo y esté optimizado de la mejor manera, tal vez con solo tres features no logremos pasar el 0.20 en testeo. 
- Cuidado con el paso anterior, porque incorporar información tiene un efecto de mejora del ajuste, lo que puede llevarnos nuevamente a overfittear. 
- Una vez que tengamos un score de testeo que nos satisfaga, todavía tenemos que ir al Kaggle y hacer la submission. Llegar a un 0.35 en testeo no implica necesariamente que en la competencia obtengamos ese resultado. Especialmente si enviciamos el modelo con mucha cross validation mal hecha. 